In [6]:
from openai import AzureOpenAI
import json
import os
import Functions
from Functions import get_weather_for_city, get_restaurants_for_city,get_hotels_for_city
import time

from dotenv import load_dotenv
load_dotenv()

azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT")
azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
api_key=os.getenv("AZURE_OPENAI_KEY")

# Create a client
client = AzureOpenAI(
    api_version="2024-05-01-preview",
    azure_endpoint=azure_endpoint,
    api_key=api_key
)

In [7]:
# Create a vector store
vector_store = client.beta.vector_stores.create(name="Contoso Hiking Products")

# Ready the files for upload to OpenAI
file_paths = ["./data/contoso/product_info_1.md", 
              "./data/contoso/product_info_2.md",
              "./data/contoso/product_info_3.md",
              "./data/contoso/product_info_4.md",
              "./data/contoso/product_info_5.md",
              "./data/contoso/product_info_6.md",
              "./data/contoso/product_info_7.md",
              "./data/contoso/product_info_8.md",
              "./data/contoso/product_info_9.md",
              "./data/contoso/product_info_10.md",
              "./data/contoso/product_info_11.md",
              "./data/contoso/product_info_12.md",
              "./data/contoso/product_info_13.md",
              "./data/contoso/product_info_14.md",
              "./data/contoso/product_info_15.md",
              "./data/contoso/product_info_16.md",
              "./data/contoso/product_info_17.md",
              "./data/contoso/product_info_18.md",
              "./data/contoso/product_info_19.md",
              "./data/contoso/product_info_20.md",
              ]
file_streams = [open(path, "rb") for path in file_paths]

# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)

In [8]:
# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)
print(vector_store.id)


completed
FileCounts(cancelled=0, completed=20, failed=0, in_progress=0, total=20)
vs_sadVq2e5jGAvR2XAhYIyNW8H


In [9]:
# Create assistant
assistant = client.beta.assistants.create(
  name="Contoso Travel Assistant",
  instructions="""
  You are a Travel assistant that provides information. 
   When asked for products, you will provide recommendations using the product_info_XX.md files provided to you. 
   You will not provide recommendations outside of those product files 
  """,
  model=azure_deployment, 
  tools=Functions.tools_function,
  tool_resources={"file_search":{"vector_store_ids":[vector_store.id]}},
  temperature=1,
  top_p=1
)

# Create thread
thread = client.beta.threads.create()
print(thread)



Thread(id='thread_u3p5jFaZoDbQXKhtwz1gDkxq', created_at=1731425707, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [10]:
prompt_prefix = """
  """

user_question ="""What hiking activities can I do in New York? 
"""

# Add a user question to the thread
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content=prompt_prefix + user_question
)

# Run the thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

# Looping until the run completes or fails
while run.status in ['queued', 'in_progress', 'cancelling']:
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

  if run.status == 'completed':
    messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  
  elif run.status == 'requires_action':
    # the assistant requires calling some functions and submit the tool outputs back to the run
    available_functions = {
      "get_weather_for_city": get_weather_for_city,
      "get_restaurants_for_city": get_restaurants_for_city,
      "get_hotels_for_city": get_hotels_for_city
    }
    tool_outputs = []
    for tool_call in run.required_action.submit_tool_outputs.tool_calls:
      function_name = tool_call.function.name
      function_to_call = available_functions[function_name]
      function_args = json.loads(tool_call.function.arguments)
          
      if function_name == "get_weather_for_city":
        function_response = get_weather_for_city(city=function_args.get("city"))
        print("retrieved weather info")
      elif function_name == "get_restaurants_for_city":
        function_response = get_restaurants_for_city(city=function_args.get("city"))
        print("retrieved restaurant info")
      elif function_name == "get_hotels_for_city":
        function_response = get_hotels_for_city(city=function_args.get("city"))
        print("retrieved hotel info")    
      else:
        function_response = "Unknown function"

      tool_outputs.append(
        { 
          "tool_call_id": tool_call.id,
          "output": str(function_response)
        }
      )
      run = client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread.id,
        run_id=run.id,
        tool_outputs = tool_outputs
      )
  
  else:
    print(run.status)


# Print the assistant response
content_block = messages.data[0].content[0]
value = content_block.text.value
print(value)

in_progress
in_progress
in_progress
in_progress
in_progress
There are various hiking activities and products you can utilize for hiking in New York:

1. **RainGuard Hiking Jacket**: A high-quality waterproof jacket that is breathable and ideal for various outdoor activities. It features an adjustable hood, multiple pockets, and ventilation zippers【4†source】【4†source】.
   
2. **TrailBlaze Hiking Pants**: Quick-drying, water-resistant, and breathable pants with articulated knees for improved mobility. They are lightweight and have multiple zippered pockets for secure storage【5†source】【5†source】.

3. **TrekStar Hiking Sandals**: These are durable and breathable hiking sandals with adjustable straps, a cushioned footbed, and excellent traction. They are suitable for hiking, walking, and other outdoor activities【2†source】【17†source】.

4. **Summit Breeze Jacket**: A lightweight and windproof jacket that is water-resistant and breathable, designed with adjustable cuffs and multiple pockets fo

In [11]:
prompt_prefix = """
  """

user_question ="""Based on the weather in New York, what products can you recommend??
"""

# Add a user question to the thread
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content=prompt_prefix + user_question
)

# Run the thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

# Looping until the run completes or fails
while run.status in ['queued', 'in_progress', 'cancelling']:
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

  if run.status == 'completed':
    messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  
  elif run.status == 'requires_action':
    # the assistant requires calling some functions and submit the tool outputs back to the run
    available_functions = {
      "get_weather_for_city": get_weather_for_city,
      "get_restaurants_for_city": get_restaurants_for_city,
      "get_hotels_for_city": get_hotels_for_city
    }
    tool_outputs = []
    for tool_call in run.required_action.submit_tool_outputs.tool_calls:
      function_name = tool_call.function.name
      function_to_call = available_functions[function_name]
      function_args = json.loads(tool_call.function.arguments)
          
      if function_name == "get_weather_for_city":
        function_response = get_weather_for_city(city=function_args.get("city"))
        print("retrieved weather info")
      elif function_name == "get_restaurants_for_city":
        function_response = get_restaurants_for_city(city=function_args.get("city"))
        print("retrieved restaurant info")
      elif function_name == "get_hotels_for_city":
        function_response = get_hotels_for_city(city=function_args.get("city"))
        print("retrieved hotel info")    
      else:
        function_response = "Unknown function"

      tool_outputs.append(
        { 
          "tool_call_id": tool_call.id,
          "output": str(function_response)
        }
      )
      run = client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread.id,
        run_id=run.id,
        tool_outputs = tool_outputs
      )
  
  else:
    print(run.status)


# Print the assistant response
content_block = messages.data[0].content[0]
value = content_block.text.value
print(value)

retrieved weather info
in_progress
in_progress
in_progress
The current weather in New York is sunny with a temperature of 10.6°C (51.1°F) and a wind speed of 19.2 mph (31 kph). Based on this weather, the following products are recommended for your hiking activities:

1. **Summit Breeze Jacket**: A lightweight, windproof, and water-resistant jacket that will keep you comfortable during your hike and protect you from the cool winds. It also features adjustable cuffs and multiple pockets【6†source】【13†source】.

2. **TrailBlaze Hiking Pants**: These quick-drying and water-resistant pants will keep you comfortable throughout your hike. They are also breathable and designed for improved mobility, which is beneficial in the current weather conditions【5†source】【5†source】.

3. **TrailLite Daypack**: A lightweight and durable daypack that is perfect for carrying essentials on your day hike. It is water-resistant and features breathable mesh panels, which are suitable for sunny weather【11†source】【

# Delete Assistant

In [12]:
response = client.beta.assistants.delete(assistant.id)